Introduction
============

This is interactive notebook regarding "Introduction to path planning". (Author: Björn Hein)

License is based on Creative Commons: Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) (pls. check: http://creativecommons.org/licenses/by-nc/4.0/)

This notebook imports all discussed algorithms and does a comparison

**Attention**: This worksheet has external dependencies 

* to the files "InnoKonz2dEnv.py" or "InnoKonzShapelyEnv.py". These files contain simple 2D environments based on numpy arrays or the shapely library with an interface for the use as collision checking in the algorithms.
* to all modules like "**Algorithm**.py", which contains the corresponding algorithms 

Important links are:

* General Info: http://www.python.org
* Python tutorial http://www.python.org/doc/tut/
* NetworkX http://networkx.github.io/
* NumPy and SciPy Guide http://docs.scipy.org/
* Matplotlib gallery http://matplotlib.sourceforge.net/gallery.html


Remember that:

* you have to press ctrl-return or shift-return, to execute the code in the code sections, only then the variables are "generated" and can be used
* you can execute the whole notebook by Cell->runAll

Adding all planners
===========


In [1]:
import sys
import HelperClass
sys.path.append("templates")
%load_ext autoreload
%autoreload 2

In [2]:
from IPPerfMonitor import IPPerfMonitor

from BasicPRM import IPVISBasicPRM 
from BasicPRM import IPBasicPRM_Roundtrip

from VisibilityPRM import IPVisibilityPRM_Roundtrip
from VisibilityPRM import IPVISVisibilityPRM
from VisibilityPRM import Optimized_IPVisibilityPRM_Roundtrip
from VisibilityPRM import CC_IPVisibilityPRM_Roundtrip

from VisibilityPRM import Normal_IPVisibilityPRM_Roundtrip

from LazyPRM import IPLazyPRM_Roundtrip
from LazyPRM import IPVISLazyPRM

from IPython.utils import io



Set-up of the test scenario and the configuration for all planner
===================================

Following a procedure to compare all discussed planners are shown:

1. Configuration for every planner is defined
2. The configuration and the planner is stored in the variable setup, a Python dict()
3. the variable setup is then used to uniformly call the planning calls


In [3]:
plannerFactory = dict()

basicConfig = dict()
basicConfig["radius"] = 3
basicConfig["numNodes"] = 200
plannerFactory["basePRM"] = [IPBasicPRM_Roundtrip.BasicPRM, basicConfig, IPVISBasicPRM.basicPRMVisualize]


#visbilityConfig = dict()
# visbilityConfig["ntry"] = 300
# plannerFactory["visibilityPRM"] = [IPVisibilityPRM_Roundtrip.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]


visbilityConfig = dict()
visbilityConfig["ntry"] = 300
plannerFactory["visibilityPRM"] = [Normal_IPVisibilityPRM_Roundtrip.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]


visbilityConfig = dict()
visbilityConfig["ntry"] = 300
plannerFactory["visibilityPRMOptimized"] = [CC_IPVisibilityPRM_Roundtrip.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]


lazyConfig = dict()
lazyConfig["initialRoadmapSize"] = 10
lazyConfig["updateRoadmapSize"]  = 5
lazyConfig["kNearest"] = 8
plannerFactory["lazyPRM"] = [IPLazyPRM_Roundtrip.LazyPRM, lazyConfig, IPVISLazyPRM.lazyPRMVisualize]

In [4]:
class ResultCollection (object):
    
    def __init__(self, plannerFactoryName, planner, benchmark, solution, perfDataFrame):
        self.plannerFactoryName = plannerFactoryName
        self.planner = planner
        self.benchmark = benchmark
        self.solution = solution
        self.perfDataFrame = perfDataFrame

In [5]:
import IPTestSuite
import importlib

In [6]:
importlib.reload(IPTestSuite)

<module 'IPTestSuite' from 'c:\\Users\\andre\\OneDrive - Hochschule Karlsruhe\\HS Karlsruhe\\1. Semester\\Roboterprogrammierung\\Projekt\\Git\\RKIM23RobotProgramDS\\02_eigeneNotebooks\\IPTestSuite.py'>

In [7]:
fullBenchList = IPTestSuite.benchList

for benchmark in fullBenchList:
    print(benchmark.name)

Trap
Bottleneck
Fat bottleneck
B_rush
circle od death


In [8]:
# List which save the time
# 1. Zeit
# 2. Nodesanzahl
# 3. Erfolgreich

sum_time_Basic = [[],[],[]]
sum_time_Visibility = [[],[],[]]
sum_time_Visibility_Optimized= [[],[],[]]
sum_time_Lazy = [[],[],[]]

list_of_benchmarks = []

In [9]:
resultList = list()
testList = fullBenchList[:5]


amountOfRepetition = 0
i = 0
while (i< amountOfRepetition):
    testList.extend(fullBenchList[:5])
    i = i + 1
 
print(testList)

for key,producer in list(plannerFactory.items()):
    HelperClass.HelperClass.printInColor(str(key)+" "+ str(producer), 'yellow')

    for benchmark in testList:
        #HelperClass.HelperClass.printInColor("Planning: " + key + " - " + benchmark.name, 'yellow')
        planner = producer[0](benchmark.collisionChecker)
        IPPerfMonitor.clearData()
        

        try:
            with io.capture_output() as captured:
                resultList.append(ResultCollection(key,
                                            planner, 
                                            benchmark, 
                                            planner.planRoundPath(benchmark.startList,benchmark.interimGoalList,benchmark.goalList,producer[1]),
                                            IPPerfMonitor.dataFrame()),)
            
            if (key == "basePRM"):
                sum_time_Basic[0].append(resultList[0].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"])                
                sum_time_Basic[1].append(len(resultList[0].planner.graph.nodes()))
                if (resultList[0].solution != []):
                    sum_time_Basic[2].append(1)
                else:
                    sum_time_Basic[2].append(0)

                
            elif (key == "visibilityPRM"):
                sum_time_Visibility[0].append(resultList[0].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"])
                
                sum_time_Visibility[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Visibility[2].append(1)
                else:
                    sum_time_Visibility[2].append(0)
            
            elif (key == "visibilityPRMOptimized"):
                sum_time_Visibility_Optimized[0].append(resultList[0].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"])
                
                sum_time_Visibility_Optimized[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Visibility_Optimized[2].append(1)
                else:
                    sum_time_Visibility_Optimized[2].append(0)


            
            elif (key == "lazyPRM"):
                sum_time_Lazy[0].append(resultList[0].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"])
                sum_time_Lazy[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Lazy[2].append(1)
                else:
                    sum_time_Lazy[2].append(0)

            list_of_benchmarks.append(benchmark.name)
            #print(resultList[0].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"])
            #print(resultList[0].solution)
            #print("Länge ", len(resultList[0].planner.graph.nodes()))
            resultList = list()
            #print(len(resultList))
                
                
            #print("Habe fertig")



        except Exception as e:
            HelperClass.HelperClass.printInColor("PLANNING ERROR ! PLANNING ERROR ! PLANNING ERROR", 'red')
            HelperClass.HelperClass.printInColor(e, 'red')
            import traceback
            HelperClass.HelperClass.printInColor(traceback.print_exc(), 'red')

            pass

        

[<IPBenchmark.Benchmark object at 0x000001BBCFD4DE90>, <IPBenchmark.Benchmark object at 0x000001BBCFCDF890>, <IPBenchmark.Benchmark object at 0x000001BBCFCDF3D0>, <IPBenchmark.Benchmark object at 0x000001BBCFDF1F90>, <IPBenchmark.Benchmark object at 0x000001BBCFDF16D0>]


KeyboardInterrupt: 

In [ ]:
print(sum_time_Lazy)
import pandas as pd
import os
from openpyxl import load_workbook

result_frames = []

print(len(sum_time_Basic[0]))

for i in range(len(sum_time_Basic[0])):
    
    # DataFrame erstellen
    data = {
        "Benchmark": [list_of_benchmarks[i]],
        "Basic Zeit": [sum_time_Basic[0][i]],
        "Basic Nodes": [sum_time_Basic[1][i]],
        "Basic Erfolgreich": [sum_time_Basic[2][i]],
        "Visibility Zeit ": [sum_time_Visibility[0][i]],
        "Visibility Nodes ": [sum_time_Visibility[1][i]],
        "Visibility Erfolgreich ": [sum_time_Visibility[2][i]],
        "Optimized_Visibility Zeit ": [sum_time_Visibility_Optimized[0][i]],
        "Optimized_Visibility Nodes ": [sum_time_Visibility_Optimized[1][i]],
        "Optimized_Visibility Erfolgreich ": [sum_time_Visibility_Optimized[2][i]],
        "LazyPRM Zeit ": [sum_time_Lazy[0][i]],
        "LazyPRM Nodes ": [sum_time_Lazy[1][i]],
        "LazyPRM Erfolgreich ": [sum_time_Lazy[2][i]],
    }

    df = pd.DataFrame(data)

    # DataFrame zur Liste hinzufügen
    result_frames.append(df)

final_df = pd.concat(result_frames, ignore_index=True)
print(type(final_df))
#df = df.append(pd.DataFrame(neue_daten), ignore_index=True)

excel_dateipfad = 'Auswertung_alle_3PRMs.xlsx'
if (os.path.isfile(excel_dateipfad)):
    print("Datei da")
    dataframe = pd.read_excel(excel_dateipfad)
    print(type(dataframe))
    dataframe = pd.concat([dataframe, final_df], ignore_index=True)
    #dataframe.append(final_df, ignore_index=True)
    


# DataFrame in eine Excel-Datei schreiben
dataframe.to_excel("Auswertung_alle_3PRMs.xlsx", index=False)


print("Habe fertig")




In [ ]:
'''
import matplotlib.pyplot as plt

for result in resultList:
    fig_local = plt.figure(figsize=(10,10))
    ax = fig_local.add_subplot(1,1,1)
    title = result.plannerFactoryName + " - " + result.benchmark.name
    if result.solution == []:
        title += " (No path found!)"
    title += "\n Assumed complexity level " + str(result.benchmark.level)
    ax.set_title(title)

    try:
        #IPVISBasicsPRM.basicPRMVisualize(result.planner, result.solution, ax=ax, nodeSize=100))
        plannerFactory[result.plannerFactoryName][2](result.planner, result.solution, ax=ax, nodeSize=100)


    except Exception as e:
        print ("Error")
        print(e)
        pass
'''    

In [ ]:
'''
import numpy as np
for bench in testList:
    title = bench.name
    pathLength = dict()
    planningTime = dict()
    roadmapSize  = dict()
    
    try:
        for result in resultList:
            if result.benchmark.name == bench.name:
                #print result.benchmark.name  + " - " +  result.plannerFactoryName, len(result.solution)
                pathLength[result.plannerFactoryName] = len(result.solution)
                planningTime[result.plannerFactoryName] = result.perfDataFrame.groupby(["name"]).sum(numeric_only=True)["time"]["planRoundPath"]
                roadmapSize[result.plannerFactoryName] = result.planner.graph.size()


        fig, ax = plt.subplots()

        width = 0.2

        ax.bar(np.arange(len(pathLength.keys())), pathLength.values(),width, color="blue")
        ax.set_ylabel(title + " Path length (number of nodes!)", color="blue")
        ax.set_xticks(np.arange(len(pathLength.keys())) + width)
        ax.set_xticklabels(pathLength.keys())

        ax2 = ax.twinx()
        bar = ax2.bar(np.arange(len(pathLength.keys()))+width, planningTime.values(),width, color="red")
        ax2.set_ylabel(title + " Planning time", color="y")

        # Add coloring and patterns on axis two
        hatches = ['x' if length==0 else '' for length in pathLength.values()]
        color   = ['red' if length==0 else 'yellow' for length in pathLength.values()]
        for i,thisbar in enumerate(bar.patches):
            thisbar.set_facecolor(color[i])
            thisbar.set_hatch(hatches[i])

        # Multiple axes 
        ax3 = ax.twinx()
        ax3.bar(np.arange(len(pathLength.keys()))+2*width, roadmapSize.values(),width, color="purple")
        ax3.set_ylabel(title + " Roadmap size",  color="purple")
        ax3.spines['right'].set_position(('axes', 1.15))
        ax3.spines['right'].set_color("purple")
    except:
        pass

'''
    
        
    

result.perfDataFrame.groupby(["name"]).sum()

In [ ]:
#result.perfDataFrame.groupby(["name"]).sum(numeric_only=True)

In [ ]:
'''
lenghBasic = 5 * (amountOfRepetition + 1)
lengthVisibility = 10 * (amountOfRepetition + 1)
#lengthLaty = 10 * amountOfRepetition


resultBasic = resultList[:lenghBasic]
resultVisibility = resultList[lenghBasic:lengthVisibility]
resultLazy = resultList[lengthVisibility:]
'''

In [ ]:
'''
import pandas as pd

result_frames = []

for i in range(len(resultBasic)):
    sum_time_Basic = resultBasic[i].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"]
    sum_time_Visibilty = resultVisibility[i].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"]
    sum_time_Lazy = resultLazy[i].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"]

    
    # DataFrame erstellen
    data = {
        "Zeit Basic": [sum_time_Basic],
        "Zeit Visibility": [sum_time_Visibilty],
        "Zeit LazyPRM" : [sum_time_Lazy]
    }

    df = pd.DataFrame(data)

    # DataFrame zur Liste hinzufügen
    result_frames.append(df)

final_df = pd.concat(result_frames, ignore_index=True)

# DataFrame in eine Excel-Datei schreiben
final_df.to_excel("Auswertung_alle_3PRMs_.xlsx", index=False)
'''


In [ ]:
#for i in range(len(resultBasic)):
#    print(resultBasic[i].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"])


In [ ]:
#for i in range(len(resultVisibility)):

#    print(resultVisibility[i].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum())


In [ ]:
#for i in range(len(resultLazy)):

#    print(resultLazy[i].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum())


In [ ]:
#result.perfDataFrame[result.perfDataFrame["name"]=="planRoundPath"][result.perfDataFrame["retVal"]==False]["time"].sum(numeric_only=True)

In [ ]:
#result.perfDataFrame[result.perfDataFrame["name"]=="planRoundPath"]["time"]

In [ ]:
#result.perfDataFrame[result.perfDataFrame["name"]=="planRoundPath"][result.perfDataFrame["retVal"]==False]["time"].sum(numeric_only=True)